## Using surprise 
- Read data
- Build basic user or item based models
- Grid Search 
- Top N users or items

In [1]:
import os
import pandas as pd
import surprise

data_dir="Z:\ML Course\Module 7 Reccomendation Engines\Data"
os.chdir(data_dir)

In [2]:
## To read a file using surprise, one needs to make sure that data is in a specific format, there are two common ways to read a dataset, to be used in the library
# Reading from dataframe
# Reading from a text file

# Reading data from a dataframe
df=pd.read_csv("sample_data.csv")
df.head()

user  rating  item
0     1       2     1
1     2       2     1
2     3       3     2
3     4       3     2
4     5       1     1

In [3]:
# Surprise expects data to have three columns, user, rating and item. The spelling of these columns names should be as given. If your dataframe has other columns or column names are differen, remember to change them before trying to reading them in surprise

# We will need to create a reader object before we can load our dataframe into surprise 
reader=surprise.dataset.Reader(line_format='user rating item',rating_scale=(1,5))

In [4]:
data=surprise.dataset.Dataset.load_from_df(df,reader=reader)

In [5]:
data.raw_ratings

[(1, 2, 1.0, None),
 (2, 2, 1.0, None),
 (3, 3, 2.0, None),
 (4, 3, 2.0, None),
 (5, 1, 1.0, None)]

In [6]:
# We can load the dataset from a text file as well, directly, just make sure the text file 
# has three columns named as user, rating and item
reader=surprise.dataset.Reader(line_format='user rating item',sep=",", 
                               rating_scale=(1,5),skip_lines=1)

In [7]:
data1=surprise.dataset.Dataset.load_from_file("sample_data.csv",reader=reader)

In [8]:
data1.raw_ratings

[('1', '1', 2.0, None),
 ('2', '1', 2.0, None),
 ('3', '2', 3.0, None),
 ('4', '2', 3.0, None),
 ('5', '1', 1.0, None)]

In [9]:
## Let's now, work with a slightly larger dataset and train memory based collaborative filtering models
data_dir="Z:\ML Course\Module 7 Reccomendation Engines\Data\ml-latest-small"
os.chdir(data_dir)

In [10]:
mr=pd.read_csv("ratings.csv")
print(mr.head())
mr.drop('timestamp',axis=1,inplace=True)
mr.rename(columns={'userId':'user','movieId':'item','rating':'rating'},inplace=True)

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [11]:
# user, item, rating on scale of 1 to 5
reader=surprise.dataset.Reader(line_format='user item rating', rating_scale=(1,5))

In [12]:
mr_train=surprise.dataset.Dataset.load_from_df(mr,reader=reader)
mr_trainset=mr_train.build_full_trainset()

In [13]:
## Create a neighbourhood based user and item based collaborative filtering model
import surprise.prediction_algorithms.knns as knns
knnbasic=knns.KNNBasic(k=40,min_k=1,sims_options={'name':'cosine','user_based':True})

In [14]:
knnbasic.train(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [15]:
mr.head()

user  item  rating
0     1    31     2.5
1     1  1029     3.0
2     1  1061     3.0
3     1  1129     2.0
4     1  1172     4.0

In [16]:
knnbasic.predict(uid=1,iid=31,r_ui=2.5)

Prediction(uid=1, iid=31, r_ui=2.5, est=2.9185383364430808, details={'actual_k': 40, 'was_impossible': False})

In [17]:
## Lets build an item based collaborative filter
knnbasic=knns.KNNBasic(k=40,min_k=1,sims_options={'name':'cosine','user_based':False})

In [18]:
knnbasic.train(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [19]:
knnbasic.predict(uid=1,iid=31)

Prediction(uid=1, iid=31, r_ui=None, est=2.9185383364430808, details={'actual_k': 40, 'was_impossible': False})

In [20]:
## Collaborative filter with average effects
knnbasic=knns.KNNWithMeans(k=40,min_k=1,sims_options={'name':'pearson','user_based':False})
knnbasic.train(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [21]:
knnbasic.predict(uid=1,iid=31)

Prediction(uid=1, iid=31, r_ui=None, est=2.0926559856597908, details={'actual_k': 40, 'was_impossible': False})

In [22]:
## Instead of  using just one train set, we can split the data into parts
# and then evaluate the model performance out of sample
mr_train.split(n_folds=3)
surprise.evaluate(knns.KNNBasic(k=40,sims_options={'name':'cosine','user_based':False}),mr_train)

Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9889
MAE:  0.7611
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9761
MAE:  0.7505
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9762
MAE:  0.7509
------------
------------
Mean RMSE: 0.9804
Mean MAE : 0.7541
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.76107589011194987,
                             0.75046295771620253,
                             0.75088328515988667],
                            'rmse': [0.9889220853279842,
                             0.97610840292552481,
                             0.97624618188519119]})

In [23]:
## Build a collaborative filter model with average effects
surprise.evaluate(knns.KNNWithMeans(k=40,sims_options={'name':'cosine','user_based':False}),mr_train)

Evaluating RMSE, MAE of algorithm KNNWithMeans.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9307
MAE:  0.7122
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9327
MAE:  0.7146
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9261
MAE:  0.7075
------------
------------
Mean RMSE: 0.9298
Mean MAE : 0.7114
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.71221295688095621,
                             0.71461915769707673,
                             0.70750320263130073],
                            'rmse': [0.93071566299109854,
                             0.93271639783111615,
                             0.92609324609596744]})

In [24]:
## Doing Grid Search
param_grid = {'k': [10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'user_based': [False]}
              }

In [25]:
algo=knns.KNNWithMeans

In [26]:
grid_search = surprise.GridSearch(algo,param_grid=param_grid, measures=['RMSE', 'MAE'])

[{'k': 10, 'sim_options': {'user_based': False, 'name': 'msd'}}, {'k': 10, 'sim_options': {'user_based': False, 'name': 'cosine'}}, {'k': 20, 'sim_options': {'user_based': False, 'name': 'msd'}}, {'k': 20, 'sim_options': {'user_based': False, 'name': 'cosine'}}]


In [27]:
grid_search.evaluate(mr_train)

------------
Parameters combination 1 of 4
params:  {'k': 10, 'sim_options': {'user_based': False, 'name': 'msd'}}
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
------------
Mean RMSE: 0.9512
Mean MAE : 0.7307
------------
------------
Parameters combination 2 of 4

C:\Users\MEDIA ENGINEER\Anaconda2\envs\tensorflow\lib\site-packages\surprise\prediction_algorithms\algo_base.py:231: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)



params:  {'k': 10, 'sim_options': {'user_based': False, 'name': 'cosine'}}
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
------------
Mean RMSE: 0.9636
Mean MAE : 0.7403
------------
------------
Parameters combination 3 of 4
params:  {'k': 20, 'sim_options': {'user_based': False, 'name': 'msd'}}
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
------------
Mean RMSE: 0.9335
Mean MAE : 0.7155
------------
------------
Parameters combination 4 of 4
params:  {'k': 20, 'sim_options': {'user_based': False, 'name': 'cosine'}}
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Do

In [28]:
print(grid_search.best_params['RMSE'])
print(grid_search.best_params['MAE'])

{'k': 20, 'sim_options': {'user_based': False, 'name': 'msd'}}
{'k': 20, 'sim_options': {'user_based': False, 'name': 'msd'}}


In [29]:
print(grid_search.best_score['RMSE'])
print(grid_search.best_score['MAE'])

0.933529241761
0.715549191233


In [30]:
## Top 5 recommendations for an item
model=knns.KNNWithMeans(k=20,sim_options={'name': 'msd', 'user_based': False})
model.train(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [31]:
mr.head()

user  item  rating
0     1    31     2.5
1     1  1029     3.0
2     1  1061     3.0
3     1  1129     2.0
4     1  1172     4.0

In [32]:
mr_trainset.to_inner_iid(1061)

2

In [33]:
model.get_neighbors(mr_trainset.to_inner_iid(1029),5)

[133, 136, 252, 357, 503]

In [34]:
for i in [133, 136, 252, 357, 503]:
   print(mr_trainset.to_raw_iid(i))

27369
50068
2102
4718
26


In [35]:
## Top 5 recommendations for a user
model=knns.KNNWithMeans(k=20,sim_options={'name': 'msd', 'user_based': True})
model.train(mr_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [36]:
mr.head()

user  item  rating
0     1    31     2.5
1     1  1029     3.0
2     1  1061     3.0
3     1  1129     2.0
4     1  1172     4.0

In [37]:
mr_trainset.to_inner_uid(1)
model.get_neighbors(mr_trainset.to_inner_uid(1),5)

[8, 25, 32, 37, 67]

In [38]:
for i in [8, 25, 32, 37, 67]:
    print(mr_trainset.to_raw_uid(i))

9
26
33
38
68
